In [12]:
import pandas as pd
import os
import json

from tqdm import tqdm
tqdm.pandas()

import random

In [10]:
train_df = pd.read_json("regnlp/ObliQADataset/ObliQA_test.json")
train_df

,QuestionID,Question,Passages,Group
0,777e7a14-fea3-4c37-a0e6-9ffb50024d5c,Can the ADGM provide clarity on the level of d...,"[{'DocumentID': 1, 'PassageID': '14.2.3.Guidan...",2
1,0eb99ea8-3810-492c-9986-7739006b5708,Are there any exceptions or specific circumsta...,"[{'DocumentID': 19, 'PassageID': '100)', 'Pass...",2
2,d34e3516-f053-4652-a0ac-ede703144b9a,What type of procedures must a Third Party Pro...,"[{'DocumentID': 3, 'PassageID': '20.14.1.(2)',...",1
3,6d876d32-7557-4149-875e-8c66f13f3485,Are there any exceptions or specific circumsta...,"[{'DocumentID': 13, 'PassageID': '4.15.16', 'P...",3
4,2efd28f4-8677-4f05-82cd-d9989fb72409,What specific areas of inventory and delivery ...,"[{'DocumentID': 34, 'PassageID': '35)', 'Passa...",3
...,...,...,...,...
2781,b7f623dc-3b61-447f-b481-d47af4115d07,Your petroleum reporting entity is finalizing ...,"[{'DocumentID': 31, 'PassageID': '10)', 'Passa...",1
2782,cd8b132c-30b0-4322-bd65-e685a5237ad9,Can the regulatory authority provide guidance ...,"[{'DocumentID': 33, 'PassageID': '113)', 'Pass...",3
2783,eb183134-d0be-4a3c-a935-7a39e9831fef,What should the detailed limit structure for a...,"[{'DocumentID': 13, 'PassageID': 'APP4.A4.1.Gu...",1
2784,c82f4441-bd41-4587-b59c-de77576370ad,In cases where an Authorised Person's internal...,"[{'DocumentID': 13, 'PassageID': 'APP6.A6.9.2....",10


In [6]:
import json
import glob

def load_json_files_from_directory(directory_path):
  """Loads all JSON files from a given directory into a list of JSON objects."""
  json_files = glob.glob(directory_path + "/*.json")
  json_data_list = []
  for json_file in json_files:
    with open(json_file, 'r') as f:
      try:
        json_data = json.load(f)
        json_data_list.append((json_file, json_data))
      except json.JSONDecodeError as e:
        print(f"Error decoding JSON in file {json_file}: {e}")
  return json_data_list


directory_path = "regnlp/ObliQADataset/StructuredRegulatoryDocuments"
json_data_list = load_json_files_from_directory(directory_path)
len(json_data_list)

40

In [7]:
flattened_json_data = [passage for json_file, json_data in json_data_list for passage in json_data]
len(flattened_json_data)

13732

In [8]:
flattened_json_data_dict = {passage["ID"]: passage for passage in flattened_json_data}

In [9]:
flattened_json_data_dict_dp_id = {f'{passage["DocumentID"]}:{passage["PassageID"]}'.replace(' ', '_'): passage for passage in flattened_json_data}

# BM25

In [2]:
!wget https://gist.githubusercontent.com/kwang2049/63ed76eb0f4d79ca81caecdb06897bfb/raw/1d86978275d666dff904fba65a34ce3e71b3cf1d/bm25.py

In [5]:
# Setup your paths appropriately
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-11.0.23.0.9-2.el7_9.x86_64/bin/java"
os.environ["JVM_PATH"] = "/usr/lib/jvm/java-11-openjdk-11.0.23.0.9-2.el7_9.x86_64/lib/server/libjvm.so"

In [4]:
from bm25 import BM25, Document, Query

bm25 = BM25()
ndocs = 40
collection = []
for i in range(1, ndocs + 1):
    with open(os.path.join("regnlp/ObliQADataset/StructuredRegulatoryDocuments", f"{i}.json")) as f:
        doc = json.load(f)
        for psg in doc:
          collection.append(Document(psg["ID"], "", psg["PassageID"] + " " + psg["Passage"]))
bm25.index(iter(collection), len(collection), "./index")

Picked up _JAVA_OPTIONS: -Xmx5g
Converting to pyserini format: 100%|███████████████████████████████████████████████████████████████████████████████████████| 13732/13732 [00:00<00:00, 109078.72it/s]


2024-11-24 01:25:41,570 INFO  [main] index.IndexCollection (IndexCollection.java:380) - Setting log level to INFO
2024-11-24 01:25:41,572 INFO  [main] index.IndexCollection (IndexCollection.java:383) - Starting indexer...
2024-11-24 01:25:41,572 INFO  [main] index.IndexCollection (IndexCollection.java:384) - ============ Loading Parameters ============
2024-11-24 01:25:41,572 INFO  [main] index.IndexCollection (IndexCollection.java:385) - DocumentCollection path: ./index/corpus
2024-11-24 01:25:41,573 INFO  [main] index.IndexCollection (IndexCollection.java:386) - CollectionClass: JsonCollection
2024-11-24 01:25:41,573 INFO  [main] index.IndexCollection (IndexCollection.java:387) - Generator: DefaultLuceneDocumentGenerator
2024-11-24 01:25:41,573 INFO  [main] index.IndexCollection (IndexCollection.java:388) - Threads: 12
2024-11-24 01:25:41,573 INFO  [main] index.IndexCollection (IndexCollection.java:389) - Language: en
2024-11-24 01:25:41,574 INFO  [main] index.IndexCollection (IndexC

In [46]:
retrieved = bm25.search(
    queries=list(Query(*tup) for tup in zip(list(train_df["QuestionID"]), list(train_df["Question"]))),
    index_path="./index",
    topk=50,
    batch_size=1,
)

SimpleSearcher class has been deprecated, please use LuceneSearcher from pyserini.search.lucene instead


Query batch: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2786/2786 [00:08<00:00, 340.09it/s]


In [ ]:
train_df["RetrievedPassages"] = train_df["QuestionID"].apply(lambda x: [flattened_json_data_dict[doc.docid] for doc in retrieved[x]])

In [48]:
triplet_data = []

for i, row in tqdm(train_df.iterrows(), total=len(train_df)):
    question_id = row["QuestionID"]
    question = row["Question"]
    positive_passages = row["Passages"]
    positive_passages = [passage["Passage"] for passage in positive_passages]
    negative_passages = list(filter(lambda x: x["Passage"] not in positive_passages, row["RetrievedPassages"]))
    negative_passages = [passage["Passage"] for passage in negative_passages]
    random_negative_passages = list(filter(lambda x: x["Passage"] not in positive_passages, random.sample(flattened_json_data, 5)))
    random_negative_passages = [passage["Passage"] for passage in random_negative_passages]
    for pos_psg in positive_passages:
        for neg_psg in negative_passages:
            triplet_data.append({
                "question": question,
                "positive_passage": pos_psg,
                "negative_passage": neg_psg,
                "type": "hard"
            })
        for neg_psg in random_negative_passages:
            triplet_data.append({
                "question": question,
                "positive_passage": pos_psg,
                "negative_passage": neg_psg,
                "type": "easy"
            })

len(triplet_data)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2786/2786 [00:00<00:00, 7196.31it/s]


197459

In [49]:
triplet_df = pd.DataFrame(triplet_data)
triplet_df

,question,positive_passage,negative_passage,type
0,Can the ADGM provide clarity on the level of d...,Relevant Persons should comply with guidance i...,The records maintained by a Relevant Person mu...,hard
1,Can the ADGM provide clarity on the level of d...,Relevant Persons should comply with guidance i...,REGULATORY REQUIREMENTS FOR AUTHORISED PERSONS...,hard
2,Can the ADGM provide clarity on the level of d...,Relevant Persons should comply with guidance i...,API REQUIREMENTS\nData\nTo enable the interope...,hard
3,Can the ADGM provide clarity on the level of d...,Relevant Persons should comply with guidance i...,API REQUIREMENTS\nData\nTo enable the interope...,hard
4,Can the ADGM provide clarity on the level of d...,Relevant Persons should comply with guidance i...,REGULATORY REQUIREMENTS - SPOT COMMODITY ACTIV...,hard
...,...,...,...,...
197454,Can you elucidate the extent to which sharehol...,Without prejudice to the Resolution Safeguards...,Requirements for all disclosures. A disclosure...,easy
197455,Can you elucidate the extent to which sharehol...,Without prejudice to the Resolution Safeguards...,The GEN rules set out the Regulator requiremen...,easy
197456,Can you elucidate the extent to which sharehol...,Without prejudice to the Resolution Safeguards...,The activities. Operating a Multilateral Tradi...,easy
197457,Can you elucidate the extent to which sharehol...,Without prejudice to the Resolution Safeguards...,A dilution levy or adjustment means a charge o...,easy


In [50]:
triplet_df.groupby("type")["question"].count()

type
easy     18327
hard    179132
Name: question, dtype: int64

In [51]:
triplet_df.to_parquet("regnlp/ObliQADataset/triplet_data_bm25_50_random_5_test.parquet")

# Q2Q

In [52]:
import torch
import torch.nn.functional as F
from tqdm import tqdm
import numpy as np
from sentence_transformers import SentenceTransformer

In [ ]:
MODEL_NAME='sentence-transformers/all-mpnet-base-v2'
model = SentenceTransformer(MODEL_NAME)

In [55]:
def get_embedding(e, task=None):
    if len(e) > 0:
        if task is not None:
            embeddings = torch.Tensor(model.encode([k.lower().split(" ") for k in e], task=task, prompt_name=task, show_progress_bar = False))
        else:
            embeddings = torch.Tensor(model.encode([k.lower() for k in e], batch_size=32, show_progress_bar = False))
        return np.array(F.normalize(embeddings, p=2, dim=1))
    else:
        return e

In [56]:
BATCH_SIZE = 1024
all_sentences = {}

for file, data in tqdm(json_data_list[0:]):

    from blingfire import text_to_sentences
    sentences = {file.split("/")[-1].split(".json")[0]+":"+e['PassageID']: text_to_sentences(e['Passage'].replace("\u200e", "").replace("\n", " ").replace("\r", " ").replace("\t", "\t")).split("\n") for e in data}
    sentences = {file.split("/")[-1].split(".json")[0]+":"+e['PassageID']: e['Passage'].replace("\u200e", "").replace("\n", " ").replace("\r", " ").replace("\t", "\t") for e in data}
    all_sentences |= sentences

0it [00:00, ?it/s]


In [57]:
if model.max_seq_length > 1024:
    model.max_seq_length = 1024

In [58]:
from itertools import chain
def compute_batch_embeddings(sentences, batch_size=32):
    output_embedding = []
    for e in tqdm(range(0, len(sentences), batch_size)):
        try:
            output_embedding.append([e for e in get_embedding(sentences[e:e+batch_size])])
        except:
            print(sentences[e:e+batch_size])
            raise Exception

    return list(chain(*output_embedding))

In [59]:
embeddings = {}
for each_sentence, embedding in zip(all_sentences.keys(), compute_batch_embeddings(list(all_sentences.values()))):
    embeddings[each_sentence] = embedding

0it [00:00, ?it/s]


In [61]:
import json
from tqdm import tqdm
import numpy as np


train_dataset_map = {}
train_dataset_ind = []
with open(f'regnlp/ObliQADataset/ObliQA_train.json', 'r') as file:
    data = json.load(file)
    questions = []
    for each_question in tqdm(data[:]):
        questions.append(each_question['Question'])
        train_dataset_map[each_question['QuestionID']] = [f"{e['DocumentID']}:{e['PassageID'].replace(' ', '_')}"for e in each_question['Passages']]
        train_dataset_ind.append(each_question['QuestionID'])

train_embeddings = np.array(compute_batch_embeddings(questions))

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 697/697 [01:02<00:00, 11.08it/s]


In [71]:
with open(f'regnlp/ObliQADataset/ObliQA_train.json', 'r') as file:
    data = json.load(file)
data

In [86]:
hack_data = []

with open(f'regnlp/ObliQADataset/ObliQA_test.json', 'r') as file:
    data = json.load(file)
    questions = []
    for each_question in tqdm(data[:]):
        questions.append(each_question['Question'])

    question_embeddings = compute_batch_embeddings(questions)

    hack_scores = {}
    for each_embedding, each_question in tqdm(zip(question_embeddings, data), total=len(data)):
        relevant_passages = each_embedding.dot(train_embeddings.T)
        indices = np.argsort(relevant_passages)

        top_10_passages = indices[-50:]
        hack_scores[each_question['QuestionID']] = relevant_passages[top_10_passages[-1]]
        i = 0
        cache_passages = set()
        for e in top_10_passages:
            for each_passage_id in train_dataset_map[train_dataset_ind[e]]:
                if each_passage_id in cache_passages:
                    continue
                cache_passages.add(each_passage_id)
                relevant_passage_dp_id_set = set([f"{e['DocumentID']}:{e['PassageID'].replace(' ', '_')}" for e in each_question['Passages']])
                hack_data.append({"question": each_question['Question'], "passage": flattened_json_data_dict_dp_id[each_passage_id]["Passage"], "similar": each_passage_id in relevant_passage_dp_id_set, "type": "hard"})
                # line = f"{each_question['QuestionID']} 0 {each_passage_id} {i+1} {relevant_passages[e]} alg"
                # pred_rels.write(line + "\n")
                i += 1

  0%|                                                                                                                                         | 0/88 [00:00<?, ?it/s]

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2786/2786 [00:07<00:00, 355.98it/s]


In [87]:
labelled_df = pd.DataFrame(hack_data)
labelled_df

,question,passage,similar,type
0,Can the ADGM provide clarity on the level of d...,An Authorised Person must implement and mainta...,False,hard
1,Can the ADGM provide clarity on the level of d...,REGULATORY REQUIREMENTS FOR AUTHORISED PERSONS...,False,hard
2,Can the ADGM provide clarity on the level of d...,REGULATORY REQUIREMENTS FOR AUTHORISED PERSONS...,False,hard
3,Can the ADGM provide clarity on the level of d...,The Regulator may make Rules prescribing circu...,False,hard
4,Can the ADGM provide clarity on the level of d...,Compliance: An Authorised Person must document...,False,hard
...,...,...,...,...
120245,Can you elucidate the extent to which sharehol...,Without prejudice to the Resolution Safeguards...,True,hard
120246,Can you elucidate the extent to which sharehol...,The Regulator may apply the Bail-in Tool only ...,False,hard
120247,Can you elucidate the extent to which sharehol...,"Subject to the section ‎31, the net proceeds o...",False,hard
120248,Can you elucidate the extent to which sharehol...,A special manager shall have all the powers of...,False,hard


In [88]:
labelled_df.to_parquet("regnlp/ObliQADataset/labelled_data_hack_50_test.parquet")

# E5

In [14]:
col_name = "ObliQA"
model_type = "e5-base-v2"

In [31]:
from qdrant_client import QdrantClient

client = QdrantClient(":memory:")

In [32]:
from qdrant_client.http.models import Distance, VectorParams, Batch
from qdrant_client.http import models


if not client.collection_exists(collection_name=col_name):
        client.create_collection(
            collection_name=col_name,
            vectors_config=VectorParams(
                size=768,
                distance=Distance.DOT
            ),
            hnsw_config=models.HnswConfigDiff(
                ef_construct=250,
                m=32
            )
        )

In [33]:
embeddings_df = pd.read_parquet("regnlp/ObliQADataset/ObliQA_e5_base_v2_finetuned_embd.parquet")
embeddings_df

,id,embd
0,60f3d71d-a67e-48ae-b737-154ebcc69c97,"[-0.22949139773845673, -0.8091111183166504, -0..."
1,0487a271-74a6-46d7-807c-3eab39adc217,"[-0.11922454833984375, -0.7714837789535522, -0..."
2,29e455a6-4550-4a6e-a2d7-94ada2de47ac,"[-0.18683482706546783, -0.7842036485671997, -0..."
3,4b126980-12f0-4e22-a36a-bfcfe99855f3,"[-0.12334126979112625, -0.8295338153839111, -0..."
4,6963b293-a008-46b6-b229-09eff7283ebe,"[-0.22615359723567963, -0.9278210401535034, -0..."
...,...,...
13727,6b4ad3d0-9455-4687-ae49-65d61e2bb31e,"[-0.1727040708065033, -0.688385009765625, -0.8..."
13728,2c33381e-7252-4748-9276-3b488e2ebebf,"[-0.21498960256576538, -0.7898073792457581, -0..."
13729,df9986be-f35e-4cae-a6c5-f9c1cf9304a7,"[-0.20870253443717957, -0.8095766305923462, -0..."
13730,985c30a3-7635-452e-ad3f-2c8c77694018,"[-0.37213513255119324, -0.7387443780899048, -0..."


In [34]:
from qdrant_client.http.models import PointStruct
import numpy as np
import tqdm 
tqdm.tqdm.pandas()

BATCH_SIZE = 5

LOAD_PRECOMPUTED = True

if LOAD_PRECOMPUTED:
    batch_list = []
    for index, row in tqdm.tqdm(embeddings_df.iterrows(), total=len(embeddings_df)):
        joined_data = flattened_json_data_dict[row["id"]]
        element_data = {
            "ID": row["id"],
            "embd": row["embd"].tolist(),
        }
        element_data["metadata"] = dict(joined_data)
        element_data["metadata"]["DocumentID"] = int(element_data["metadata"]["DocumentID"])
        element_data["metadata"]["ID"] = row["id"]
        batch_list.append(element_data)
        if index % BATCH_SIZE == 0:
            client.upsert(
                collection_name=col_name,
                wait=True,
                points=[
                    PointStruct(id=element["ID"], vector=element["embd"], payload=element["metadata"])
                    for element in batch_list
                ],
            )
            batch_list = []
    if len(batch_list) > 0:
        client.upsert(
                collection_name=col_name,
                wait=True,
                points=[
                    PointStruct(id=element["ID"], vector=element["embd"], payload=element["metadata"])
                    for element in batch_list
                ],
            )

  0%|                                                                                                                                      | 0/13732 [00:00<?, ?it/s]

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 13732/13732 [00:03<00:00, 4499.85it/s]


In [64]:
test_df = pd.read_json("regnlp/ObliQADataset/ObliQA_test.json")
test_df

,QuestionID,Question,Passages,Group
0,777e7a14-fea3-4c37-a0e6-9ffb50024d5c,Can the ADGM provide clarity on the level of d...,"[{'DocumentID': 1, 'PassageID': '14.2.3.Guidan...",2
1,0eb99ea8-3810-492c-9986-7739006b5708,Are there any exceptions or specific circumsta...,"[{'DocumentID': 19, 'PassageID': '100)', 'Pass...",2
2,d34e3516-f053-4652-a0ac-ede703144b9a,What type of procedures must a Third Party Pro...,"[{'DocumentID': 3, 'PassageID': '20.14.1.(2)',...",1
3,6d876d32-7557-4149-875e-8c66f13f3485,Are there any exceptions or specific circumsta...,"[{'DocumentID': 13, 'PassageID': '4.15.16', 'P...",3
4,2efd28f4-8677-4f05-82cd-d9989fb72409,What specific areas of inventory and delivery ...,"[{'DocumentID': 34, 'PassageID': '35)', 'Passa...",3
...,...,...,...,...
2781,b7f623dc-3b61-447f-b481-d47af4115d07,Your petroleum reporting entity is finalizing ...,"[{'DocumentID': 31, 'PassageID': '10)', 'Passa...",1
2782,cd8b132c-30b0-4322-bd65-e685a5237ad9,Can the regulatory authority provide guidance ...,"[{'DocumentID': 33, 'PassageID': '113)', 'Pass...",3
2783,eb183134-d0be-4a3c-a935-7a39e9831fef,What should the detailed limit structure for a...,"[{'DocumentID': 13, 'PassageID': 'APP4.A4.1.Gu...",1
2784,c82f4441-bd41-4587-b59c-de77576370ad,In cases where an Authorised Person's internal...,"[{'DocumentID': 13, 'PassageID': 'APP6.A6.9.2....",10


In [36]:
import pandas as pd
import torch
import torch.nn.functional as F

from torch import Tensor
from transformers import AutoTokenizer, AutoModel

In [37]:
device = "cuda:0"
MODEL_NAME = "regnlp/e5-base-v2-finetuned/checkpoint-24800"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModel.from_pretrained(MODEL_NAME).to(device)

In [38]:
def average_pool(last_hidden_states: Tensor,
                 attention_mask: Tensor) -> Tensor:
    last_hidden = last_hidden_states.masked_fill(~attention_mask[..., None].bool(), 0.0)
    return last_hidden.sum(dim=1) / attention_mask.sum(dim=1)[..., None]

In [ ]:
vec_data = []

BATCH_SIZE = 8
TOP_K = 50

pred_rels = open("test.pred_rels", "w")

batched_data = np.array_split(np.asarray(list(test_df.iterrows()), dtype="object"), len(test_df) // BATCH_SIZE)
for batched_row in tqdm.tqdm(batched_data, total=len(batched_data)):
    doc_batch_dict = tokenizer([element[1]['Question'] for element in batched_row], padding=True, truncation=True, return_tensors='pt')
    for key in doc_batch_dict:
        doc_batch_dict[key] = doc_batch_dict[key].to(device)
    query_embeddings_list = None
    with torch.no_grad():
        doc_outputs = model(**doc_batch_dict)
        # doc_embeddings = last_token_pool(doc_outputs.last_hidden_state, doc_batch_dict['attention_mask'])
        query_embeddings_list = average_pool(doc_outputs.last_hidden_state, doc_batch_dict['attention_mask'])
        for key in doc_batch_dict:
            doc_batch_dict[key] = doc_batch_dict[key].detach().cpu()
    for i in range(len(query_embeddings_list)):
        search_result = client.query_points(
            collection_name=col_name,
            query=query_embeddings_list[i].tolist(),
            with_payload=True,
            limit=TOP_K,
            # search_params=models.SearchParams(hnsw_ef=128, exact=False),
            search_params=models.SearchParams(exact=True),
        ).points
        relevant_passage_dp_id_set = set([f"{e['DocumentID']}:{e['PassageID'].replace(' ', '_')}" for e in batched_row[i][1]["Passages"]])
        for j, hit in enumerate(sorted(search_result, key=lambda hit:hit.score, reverse=True)):
            vec_data.append({"question": batched_row[i][1]["Question"], "passage": hit.payload["Passage"], 
            "similar": f"{hit.payload['DocumentID']}:{hit.payload['PassageID'].replace(' ', '_')}" in relevant_passage_dp_id_set, 
            "type": "hard"})
            pred_rels.write(f"{batched_row[i][1]['QuestionID']} 0 {hit.id} {j+1} 10 alg\n")

pred_rels.close()

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 348/348 [01:11<00:00,  4.89it/s]


In [3]:
!regnlp/trec_eval/trec_eval -m recall.10 -m map_cut.10 regnlp/test_gt.trec regnlp/test.pred_rels

In [61]:
labelled_df = pd.DataFrame(vec_data)
labelled_df

,question,passage,similar,type
0,If a PFP Client intends to use the exit facili...,DIGITAL SECURITIES – INTERMEDIARIES\nThis sect...,False,hard
1,If a PFP Client intends to use the exit facili...,MARKETS RULES – OFFERS OF SECURITIES TO THE PU...,False,hard
2,If a PFP Client intends to use the exit facili...,Review Procedures for Identifying Entity Accou...,False,hard
3,If a PFP Client intends to use the exit facili...,All Regulatory Returns prepared by the Authori...,False,hard
4,If a PFP Client intends to use the exit facili...,DUE DILIGENCE FOR PRE-EXISTING INDIVIDUAL ACCO...,False,hard
...,...,...,...,...
139395,Can a Recognised Body rely solely on its inter...,REGULATORY REQUIREMENTS\nData Protection\nFor ...,False,hard
139396,Can a Recognised Body rely solely on its inter...,CRCOM. An Authorised Person must calculate its...,False,hard
139397,Can a Recognised Body rely solely on its inter...,"Subject to paragraphs C through E, each Report...",False,hard
139398,Can a Recognised Body rely solely on its inter...,A Relevant Person must ensure that it does not...,False,hard


In [63]:
labelled_df.to_parquet("regnlp/ObliQADataset/labelled_data_e5_finetuned_50_dev.parquet")

# Combine All sources

After generating data for all sources and all splits (train, test, dev) of the ObliQADataset. We can combine all the sources and splits into a single files for splits

We skipped BGE due to less compute constraints

In [ ]:
"""
labelled_data_bm25_100_random_5.parquet
labelled_data_bm25_100_random_5_dev.parquet
labelled_data_bm25_100_random_5_test.parquet
labelled_data_e5_finetuned_50.parquet
labelled_data_e5_finetuned_50_dev.parquet
labelled_data_e5_finetuned_50_test.parquet
labelled_data_hack_50.parquet
labelled_data_hack_50_dev.parquet
labelled_data_hack_50_test.parquet
"""

# combine dev, test and other parquet files as 3 different dataframes
labelled_data_dev = pd.concat([
    pd.read_parquet("regnlp/ObliQADataset/labelled_data_bm25_100_random_5_dev.parquet"),
    pd.read_parquet("regnlp/ObliQADataset/labelled_data_e5_finetuned_50_dev.parquet"),
    pd.read_parquet("regnlp/ObliQADataset/labelled_data_hack_50_dev.parquet")
    ])
labelled_data_test = pd.concat([
    pd.read_parquet("regnlp/ObliQADataset/labelled_data_bm25_100_random_5_test.parquet"),
	pd.read_parquet("regnlp/ObliQADataset/labelled_data_e5_finetuned_50_test.parquet"),
	pd.read_parquet("regnlp/ObliQADataset/labelled_data_hack_50_test.parquet")
	])
labelled_data_other = pd.concat([
    pd.read_parquet("regnlp/ObliQADataset/labelled_data_bm25_100_random_5.parquet"),
	pd.read_parquet("regnlp/ObliQADataset/labelled_data_e5_finetuned_50.parquet"),
	pd.read_parquet("regnlp/ObliQADataset/labelled_data_hack_50.parquet")
	])

# pretty print lengths of each data frames
print("dev: ", len(labelled_data_dev))
print("test: ", len(labelled_data_test))
print("other: ", len(labelled_data_other))

In [ ]:
# remove duplicates if types are different
labelled_data_dev = labelled_data_dev.drop_duplicates(subset=["question", "passage", "similar"])
labelled_data_test = labelled_data_test.drop_duplicates(subset=["question", "passage", "similar"])
labelled_data_other = labelled_data_other.drop_duplicates(subset=["question", "passage", "similar"])

# pretty print lengths of each data frames
print("dev: ", len(labelled_data_dev))
print("test: ", len(labelled_data_test))
print("other: ", len(labelled_data_other))

In [ ]:
labelled_data_dev = labelled_data_dev.drop_duplicates(subset=["question", "passage", "similar"])
labelled_data_test = labelled_data_test.drop_duplicates(subset=["question", "passage", "similar"])
labelled_data_other = labelled_data_other.drop_duplicates(subset=["question", "passage", "similar"])

print("dev: ", len(labelled_data_dev))
print("test: ", len(labelled_data_test))
print("other: ", len(labelled_data_other))

In [ ]:
labelled_data_dev = labelled_data_dev.drop_duplicates(subset=["question", "passage"])
labelled_data_test = labelled_data_test.drop_duplicates(subset=["question", "passage"])
labelled_data_other = labelled_data_other.drop_duplicates(subset=["question", "passage"])

print("dev: ", len(labelled_data_dev))
print("test: ", len(labelled_data_test))
print("other: ", len(labelled_data_other))

In [ ]:
# save to parquet
labelled_data_dev.to_parquet("regnlp/ObliQADataset/reg_nlp_labelled_data_dev.parquet")
labelled_data_test.to_parquet("regnlp/ObliQADataset/reg_nlp_labelled_data_test.parquet")
labelled_data_other.to_parquet("regnlp/ObliQADataset/reg_nlp_labelled_data_train.parquet")